# Franka Robot Sequential Object Manipulation

This example demonstrates a Franka Emika Panda robot sequentially manipulating multiple objects. The robot follows a structured plan:

1. Start from a ready position
2. Move above an object from the list
3. Grasp the object
4. Execute a planned movement with the object
5. Release the object
6. Move 10 cm above the current position
7. Return to ready position
8. Repeat for the next object in the list until all objects are manipulated

In [ ]:
import genesis as gs
import numpy as np
import torch
from pytransform3d import (
    transformations as pt,
    rotations as pr,
    batch_rotations as pb,
    trajectories as ptr,
    plot_utils as ppu
)
from pandaSim.geometry.genesis_adapter import GenesisAdapter
from pandaSim.planning.screw_motion_planner import ScrewMotionPlanner

import spatialmath as sm
# auto reload
%load_ext autoreload
%autoreload 2

In [ ]:
gs.destroy()
gs.init(backend=gs.cpu)
gs.set_random_seed(seed=42)
adapter = GenesisAdapter()
planner = ScrewMotionPlanner()

## Scene Construction

Here we create the simulation environment with:
1. A ground plane
2. A Franka Emika Panda robot with spinning pads and virtual finger
3. Multiple objects (cube, bottle, cylinder) that will be processed sequentially
4. A target entity for visualization

In [ ]:
scene = adapter.scene
plane = scene.add_entity(
    gs.morphs.Plane(),
)

franka = scene.add_entity(
    gs.morphs.MJCF(file="../assets/xml/franka_emika_panda/panda_vir.xml")
)

# Create a list to hold all objects for sequential processing
objects_list = []

table1 = scene.add_entity(
    gs.morphs.Box(
        size = (0.7, 1.7, 0.02),
        pos = (0.5, 0, 0.18),
        fixed = True,
    ),
    surface=gs.surfaces.Default(
        color = (0.5, 0.5, 0.5),
    ),
    material=gs.materials.Rigid(
        gravity_compensation=1.0 ,
        friction=5.0
    )
)
        
table2 = scene.add_entity(
    gs.morphs.Box(
        size = (0.7, 1.7, 0.02),
        pos = (-0.5, 0, 0.18),
        fixed = True,
    ),
    surface=gs.surfaces.Default(
        color = (0.5, 0.5, 0.5),
    ),
    material=gs.materials.Rigid(
        gravity_compensation=1.0,
        friction=5.0
    )
)

table3 = scene.add_entity(
    gs.morphs.Box(
        size = (0.3, 0.7, 0.02),
        pos = (0, -0.5, 0.18),
        fixed = True,
    ),
    surface=gs.surfaces.Default(
        color = (0.5, 0.5, 0.5),
    ),
    material=gs.materials.Rigid(
        gravity_compensation=1.0,
        friction=5.0
    )
)

table4 = scene.add_entity(
    gs.morphs.Box(
        size = (0.3, 0.7, 0.02),
        pos = (0, 0.5, 0.18),
        fixed = True,
    ),
    surface=gs.surfaces.Default(
        color = (0.5, 0.5, 0.5),
    ),
    material=gs.materials.Rigid(
        gravity_compensation=1.0,
        friction=5.0
    )
)

# Object 1: Cube
cube_size = (0.1, 0.07, 0.25)
cube_pos = (0.55, 0, 0.05+0.19)
cube = scene.add_entity(
    gs.morphs.Box(
        size=cube_size,
        pos=cube_pos,
        euler=(0, -90, 0)
    ),
    surface=gs.surfaces.Default(
        color=(0.5, 0.8, 0.94),
    ),
    material=gs.materials.Rigid(friction=5)
)


# Object 2: Bottle
bottle = scene.add_entity(
    material=gs.materials.Rigid(rho=300),
    morph=gs.morphs.URDF(
        file="urdf/3763/mobility_vhacd.urdf",
        scale=0.09,
        pos=(0.65, 0.3, 0.036+0.19),
        euler=(30, -90, 0),
    ),
)


# Object 3: Cylinder
cylinder_pos = (0.7, -0.3, 0.03+0.19)
cylinder_euler = (-30, -90, 0)
cylinder_radius = 0.03
cylinder_height = 0.15
cylinder = scene.add_entity(
    gs.morphs.Cylinder(
        radius=cylinder_radius,
        height=cylinder_height,
        pos=cylinder_pos,
        euler=cylinder_euler,
    ),
    surface=gs.surfaces.Default(
        color=(0.1, 0.8, 0.1),
    )
)


# Target entity for visualization
target_entity = scene.add_entity(
    gs.morphs.Mesh(
        file="meshes/axis.obj",
        scale=0.15,
        collision=False,
    ),
    surface=gs.surfaces.Default(color=(1, 0.5, 0.5, 1)),
    material=gs.materials.Rigid(gravity_compensation=1.0)
)

## Robot Configuration

We build the scene and define important joint groups:
- `motors_dof`: Joints 0-6 for arm movement
- `fingers_dof`: Joints 7-8 for gripper fingers
- `virtual_finger`: Joint 9 for rotational slippage
- `spinning_pads`: Joints 10-11 for spinning pads

In [ ]:
scene.build(compile_kernels=False)

# Define joint groups
motors_dof = np.arange(7)  # Joints 0-6 for arm movement
fingers_dof = np.arange(7, 9)  # Joints 7-8 for gripper fingers
virtual_finger_idx = 9  # Joint 9 for virtual finger
spinning_pads = np.arange(10, 12)  # Joints 10-11 for spinning pads

## Initial Robot Positioning

Set up the robot in its ready pose, which will be the starting and returning position for each object manipulation cycle.

In [33]:
# Get end effector and virtual finger links
end_effector = franka.get_link('hand')
virtual_finger = franka.get_link('virtual_finger')
def reset_scene():
    scene.reset()

    # Define ready pose joint positions
    qr = np.array([0.0000, -0.3002, 0.0000, -2.1991, 0.0000, 2.0071, 0.7854, 0.04, 0.04])
    ready_qpos = np.append(qr, [0, 0, 0])  # Adding positions for virtual finger and spinning pads

    # Set robot to ready pose
    franka.ready_qpos = ready_qpos.squeeze()
    franka.set_qpos(franka.ready_qpos)
    franka.control_dofs_position(franka.ready_qpos)
    
    # Visualize initial position
    target_entity.set_pos(virtual_finger.get_pos())
    target_entity.set_quat(virtual_finger.get_quat())

    scene.step()

## Helper Functions

Define functions to handle the different steps of our manipulation plan.

In [34]:
def move_to_position(robot, target_link, position, orientation, fingers_state=0.04):
    """Move the robot to a specific position and orientation"""
    qpos = robot.inverse_kinematics(
        link=target_link,
        pos=position,
        quat=orientation,
        pos_tol=1e-5,
        rot_mask=[False, False, True]
    )
    qpos[fingers_dof] = fingers_state
    
    path = robot.plan_path(
        qpos_goal=qpos,
        num_waypoints=250  # 2s duration
    )
    
    # Execute the planned path
    for waypoint in path:
        robot.control_dofs_position(waypoint)
        scene.step()
    
    # Allow robot to reach the last waypoint
    adapter.step_simulation(1)
    
    return qpos

def compute_object_grasp(obj, grasp_height='top', offset_toward=0.04, offset_upward=0.02):
    """Compute a grasp pose for the given object"""
    scene.step()
    
    # Define gripper orientation for grasping
    grasp_T_gripper = np.array([
        [0, 1, 0, 0],
        [0, 0, -1, 0],
        [-1, 0, 0, 0],
        [0, 0, 0, 1]
    ])
    
    # Compute grasp pose
    grasp_pose, qs, s_axes = planner.compute_grasp(
        obj=obj,
        adapter=adapter,
        prefer_closer_grasp=True,
        grasp_height=grasp_height,
        offset_toward=offset_toward, 
        offset_upward=offset_upward,
        gripper_offset=grasp_T_gripper,
        output_type='pq'
    )
    
    # Visualize grasp pose
    target_entity.set_qpos(grasp_pose)
    scene.step()
    
    return grasp_pose, qs, s_axes

def grasp_object(robot, finger_link, grasp_pose):
    """Move to grasp position and close the gripper"""
    # Move to pre-grasp position (10cm above grasp position)
    pregrasp_pos = grasp_pose[:3] + np.array([0, 0, 0.1])
    move_to_position(robot, finger_link, pregrasp_pos, grasp_pose[3:])
    
    # Move down to grasp position
    move_to_position(robot, finger_link, grasp_pose[:3], grasp_pose[3:])
    
    # Close gripper
    robot.control_dofs_force(np.array([-2, -2]), fingers_dof)
    adapter.step_simulation(1)

def execute_manipulation(robot, finger_link, obj, qs, s_axes):
    """Execute planned manipulation with the grasped object"""
    # Plan a manipulation trajectory
    trajectory = planner.plan(
        robot=robot,
        link=finger_link,
        object=obj,
        adapter=adapter,
        qs=qs,
        s_axes=s_axes,
        output_type='pq'
    )
    
    # Convert trajectory to joint positions
    qposs = []
    qpos = robot.get_qpos()
    for pq in trajectory:
        qpos = robot.inverse_kinematics(
            link=finger_link,
            pos=pq[:3],
            quat=pq[3:],
            init_qpos=qpos,
            pos_tol=1e-5,
            rot_mask=[False, False, True]
        )
        qposs.append(qpos)
    
    # Execute trajectory
    for waypoint in qposs:
        robot.control_dofs_position(waypoint[motors_dof], motors_dof)
        target_entity.set_pos(finger_link.get_pos())
        target_entity.set_quat(finger_link.get_quat())
        scene.step()

def release_object(robot):
    """Open the gripper to release the object"""
    robot.control_dofs_force(np.array([0, 0]), fingers_dof)
    robot.control_dofs_position(np.array([0.04, 0.04]), fingers_dof)
    adapter.step_simulation(1)

def move_up_from_current(robot, finger_link, height=0.1):
    """Move up by specified height from current position"""
    current_pose = adapter.forward_kinematics(robot, finger_link, output_type='pq')
    target_pos = current_pose[:3] + np.array([0, 0, height])
    move_to_position(robot, finger_link, target_pos, current_pose[3:])

def return_to_ready_pose(robot):
    """Return the robot to its ready pose"""
    path = robot.plan_path(
        qpos_goal=robot.ready_qpos,
        num_waypoints=250
    )
    
    for waypoint in path:
        robot.control_dofs_position(waypoint)
        scene.step()
    
    adapter.step_simulation(1)

## Main Execution Loop

Now we'll process each object in the list sequentially, following our plan:
1. Move to above the object
2. Grasp the object
3. Execute the manipulation plan
4. Release the object
5. Move 10cm up
6. Return to ready pose
7. Process next object

In [35]:
objects_list = []
objects_list.append((cube, "cube"))
objects_list.append((cylinder, "cylinder"))
objects_list.append((bottle, "bottle"))
reset_scene()
# Process each object in the list
while objects_list:
    # Get the next object from the list
    obj, obj_name = objects_list.pop(0)
    print(f"\nProcessing {obj_name}...")
    
    # Step 1: Compute grasp pose for the object
    print(f"Computing grasp pose for {obj_name}")
    grasp_height = 'center' if obj_name == 'cylinder' else 'top'
    offset_toward = 0.04 if obj_name == 'bottle' else 0.02
    offset_upward = 0.02
    grasp_pose, qs, s_axes = compute_object_grasp(obj=obj, 
                                                  grasp_height=grasp_height, 
                                                  offset_toward=offset_toward, 
                                                  offset_upward=offset_upward)
    
    # Step 2: Grasp the object
    print(f"Moving to grasp {obj_name}")
    grasp_object(franka, virtual_finger, grasp_pose)
    
    # Step 3: Execute manipulation plan
    print(f"Executing manipulation with {obj_name}")
    execute_manipulation(franka, virtual_finger, obj, qs, s_axes)
    
    # Step 4: Release the object
    print(f"Releasing {obj_name}")
    release_object(franka)
    
    # Step 5: Move 10cm up from the current position
    print(f"Moving up from {obj_name}")
    move_up_from_current(franka, virtual_finger, 0.1)
    
    # Step 6: Return to ready pose
    print(f"Returning to ready pose")
    return_to_ready_pose(franka)
    
    print(f"Completed processing {obj_name}")

print("\nAll objects processed successfully!")

[Genesis] [01:16:13] [INFO] Resetting Scene <10582e4>.



Processing cube...
Computing grasp pose for cube
Moving to grasp cube
[Genesis] [01:16:13] [INFO] Path solution found successfully.
[Genesis] [01:16:19] [INFO] Path solution found successfully.
Executing manipulation with cube
Releasing cube
Moving up from cube
[Genesis] [01:16:43] [INFO] Path solution found successfully.
Returning to ready pose
[Genesis] [01:16:49] [INFO] Path solution found successfully.
Completed processing cube

Processing cylinder...
Computing grasp pose for cylinder
Moving to grasp cylinder
[Genesis] [01:16:56] [INFO] Path solution found successfully.
[Genesis] [01:17:02] [WARNING] `qpos_start` exceeds joint limit. Relaxing joint limit to contain `qpos_start` for planning.
[Genesis] [01:17:02] [INFO] Path solution found successfully.
Executing manipulation with cylinder
theta changed to  -1.5707963267948966
Releasing cylinder
Moving up from cylinder
[Genesis] [01:17:20] [WARNING] `qpos_start` exceeds joint limit. Relaxing joint limit to contain `qpos_start` for 

In [31]:
adapter.get_size(bottle)

array([0.039, 0.068, 0.167])

In [45]:
reset_scene()


[Genesis] [01:29:18] [INFO] Resetting Scene <10582e4>.


In [47]:
adapter.get_size(cylinder)

array([0.06, 0.06, 0.15])

In [50]:
obb = adapter.get_obb(cylinder)

obb["center"]

array([ 0.70000059, -0.30000013,  0.21958356])

In [46]:
qs, s_axes = planner.screw_from_bbox(adapter.get_obb(cylinder))

qs, s_axes 

[array([False, False,  True,  True])]
[[0.02633112 0.04560456 0.02814807]
 [0.02633112 0.04560456 0.02814807]]
[[5 7]
 [4 6]]


([array([[ 0.64208544, -0.25031085,  0.19325539],
         [ 0.77198876, -0.32531189,  0.19325148]])],
 [array([[0.44097513, 0.76375324, 0.4714042 ],
         [0.44097513, 0.76375324, 0.4714042 ]])])

In [42]:
for verts in adapter.get_obb(cylinder)["vertices"]:
    target_entity.set_pos(verts)
    adapter.step_simulation(1)

In [41]:
adapter.get_obb(cylinder)

{'vertices': array([[ 1.03780857,  0.01369277,  0.06496037],
        [ 0.91182538, -0.06772147,  0.06496024],
        [ 1.06959473, -0.03549421,  0.05331139],
        [ 0.94361154, -0.11690845,  0.05331125],
        [ 1.031486  ,  0.02347662,  0.00639659],
        [ 0.90550281, -0.05793761,  0.00639646],
        [ 1.06327216, -0.02571036, -0.00525239],
        [ 0.93728897, -0.10712459, -0.00525253]]),
 'edges': array([[ 6.32256909e-03, -9.78385871e-03,  5.85637792e-02],
        [ 6.32256909e-03, -9.78385871e-03,  5.85637792e-02],
        [ 1.25983194e-01,  8.14142379e-02,  1.35828609e-07],
        [ 1.25983194e-01,  8.14142379e-02,  1.35828609e-07],
        [ 6.32256909e-03, -9.78385871e-03,  5.85637792e-02],
        [ 6.32256909e-03, -9.78385871e-03,  5.85637792e-02],
        [ 1.25983194e-01,  8.14142379e-02,  1.35828609e-07],
        [ 1.25983194e-01,  8.14142379e-02,  1.35828609e-07],
        [ 3.17861611e-02, -4.91869802e-02, -1.16489863e-02],
        [ 3.17861611e-02, -4.9186980